# *Machine learning* et malédiction de la dimension

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être capable de différencier l'apprentissage supervisé et non supervisé.
* Être initié à&nbsp;:
    * Construire un modèle à partir d'un jeu de données,
    * Évaluer ce modèle,
    * Le sur-apprentissage,
    * La malédiction de la dimension.
* Manipuler&nbsp;:
    * Quelques modèles standards de la librairie $\texttt{sklearn}$.



 ----

## I. Introduction

Imaginons que nous souhaitions construire une application qui prendrait en entrée une image de chien ou de chat et qui doive prédire laquelle des deux espèces est représentée. Imaginons encore une application qui prendrait en entrée un mail qu'elle classifierait comme SPAM ou NONSPAM. Supposons qu'il existe deux catégories de clients qu'on ne connait pas *a priori* et que l'entreprise souhaite prédire pour chacun des clients sa catégorie. On peut vouloir prédire la température qu'il fera demain à partir de données relevées aujourd'hui.

Une constante est partagée par l'ensemble de ces scénarios. Il y a tout d'abord une donnée d'entrée plus ou moins complexe et structurée. On notera $\mathcal{X}$ l'espace auquel elle appartient. Ensuite, à partir de cette donnée, l'objectif est de faire une prédiction. Notons $\mathcal{Y}$ l'espace auquel appartient cette prédiction. On appelle ça aussi nos labels ou nos variables à expliquer. Notre objectif, en tant que *machine learner* est de construire une fonction $h:\mathcal{X}\mapsto\mathcal{Y}$ qui aura de *bonnes performances* "en production", c'est-à-dire sur des données nouvelles que nous n'avons jamais vues (i.e. on ne veut pas prédire la météo d'hier à partir d'avant hier, mais bien de demain à partir d'aujourd'hui). 

Deux types d'apprentissage sont généralement opposés&nbsp;: l'apprentissage supervisé (AS) et non-supervisé (ANS).

### A. L'apprentissage supervisé

L'apprentissage supervisé part du principe que (1) nos labels (i.e. l'espace $\mathcal{Y}$) est bien défini et (2) que nous avons accès à des données associant des éléments de $\mathcal{X}$ à leur label $\mathcal{Y}$.

On parlera de problème de régression si par exemple $\mathcal{Y}\subseteq\mathbb{R}$ ou de problème de classification si $\mathcal{Y}=\{1, \ldots, C\}$ où l'ordre n'est pas important. Par exemple, prédire la température est un problème de régression alors que prédire si la photo représente un chien ou un chat est un problème de classification.

A fortiori, toutes les observations dans $\mathcal{X}$ ne sont pas nécessairement équiprobables. Certains clients ont peut-être, par exemple, un profil plus commun que d'autres. Afin de pouvoir définir plus rigoureusement ce qu'on entend par *bonnes performances*, notons $X\in\mathcal{X}$ une variable aléatoire qui décrit nos données observées et $Y\in\mathcal{Y}$ la variable aléatoire associée à nos labels. Assez naïvement, notons $\mathbb{P}$ la loi de notre couple $X,Y$&nbsp;:

$$X, Y\sim \mathbb{P}.$$

Notons $r:\mathcal{Y}\times\mathcal{Y}\rightarrow\mathbb{R}^+$ une mesure d'erreur, un risque élémentaire. On a par exemple, dans le cas d'un problème de régression, l'erreur quadratique&nbsp;:

$$r(\hat{y}, y)=(\hat{y}-y)^2,$$

où $\hat{y}$ est la prédiction que ferait notre modèle. Ou encore, dans le cas de la classification nous pouvons avoir cette fois-ci l'erreur $0.1$&nbsp;:

$$r(\hat{y}, y)=\textbf{1}\{\hat{y}\neq y\},$$

qui vaut $1$ si la prédiction est mauvaise ou $0$ sinon.

Notre objectif est tout naturellement de trouver une application $h:\mathcal{X}\mapsto\mathcal{Y}$ telle que $R(h)=\mathbb{E}\big[r(h(X), Y)\big]$ est petit. On veut un bon modèle sur de nouvelles données. L'idée va être de collecter des données représentatives (dans le sens iid) et de construire notre modèle avec ces dernières. Notons&nbsp;:

$$S_n=\{(X_i, Y_i)\}_{i\leq n}$$

un jeu de données de taille $n$.

### B. L'apprentissage non-supervisé

Ici, c'est l'inverse. Nous avons accès à l'espace des observations $\mathcal{X}$ duquel on peut collecter des données (toujours selon la loi de la variable aléatoire $X$). On sait qu'il existe un espace $\mathcal{Y}$ cible mais (1) il n'est pas nécessairement connu et (2) nous ne connaissons pas d'exemple de liens entre exemples d'apprentissage et cibles associées. 

Par exemple, si $\mathcal{X}$ représente des données clients, on peut savoir (se douter) qu'il existe des groupes de clients qui se ressemblent mais ne pas les connaître et ne pas savoir combien il y en a. Il s'agit ici d'une tâche de *clustering* où on cherche à regrouper des données entre-elles toujours de manière à ce que le regroupement se généralise à de nouvelles données.

On peut chercher à transformer nos données dans $\mathcal{X}$ dans un espace qu'on notera cette fois-ci $\mathcal{Z}$ où ces dernières auront de meilleures propriété. On note cet "espace de représentation" $\mathcal{Z}$ et non $\mathcal{Y}$ car il s'agit souvent d'une étape intermédiaire avant une tâche supervisée où on chercherait à prédire un label dans $\mathcal{Y}$. C'est ce qu'on appelle l'apprentissage de représentation. Ainsi, si $\mathcal{X}$ est l'ensemble des photos de chiens et de chats, $\mathcal{Z}$ est l'ensemble de ces dernières où on a mis "d'un côté" les photos de chiens et de "l'autre" celles de chats. Il devient simple de construire une tâche supervisée permettant de prédire le bon label "chien/chat".

## II. On casse une idée préconçue (AS)

Soit $S=\{(x_i, y_i)\}_{i\leq n}$ un jeu de données représentatif de taille $n$. Un modèle très performant sur ces données est-il performant sur des données nouvelles ? Réfléchissez quelques instants et écrivez votre réponse dans un coin.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics

import numpy as np

import matplotlib.pyplot as plt

Chargeons et affichons notre jeu de données. Ce dernier consiste en des chiffres écrits à la main. L'objectif va être de faire un modèle qui permet de prédire ces derniers.

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

Construisons notre premier modèle. Ce dernier correspond à la fonction mathématique suivante&nbsp;: 


$$\begin{aligned}h(x)=\begin{cases}y&\text{ si } (x, y)\in S_n\\\text{aléatoire}()&\text{ sinon.}\end{cases}\end{aligned}$$

On retourne le label connu si on a déjà vu le "point" et on retourne un label au hasard sinon.

In [ ]:
class Memorize(object):
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.X, self.y = X, y
        
    def predict(self, X):
        y = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            memorized = False
            for j in range(self.X.shape[0]):
                # On compare les pixels un a un
                # et on regarde s'ils sont tous identiques
                # pixel wise comparison
                if (self.X[j] == X[i]).sum() == X.shape[1]:
                    y[i] = self.y[j]
                    memorized = True
                    break
            if not memorized:
                y[i] = np.random.randint(0, 10)
        return y

Le code suivant prépare nos données (des images) afin qu'elles deviennent facilement manipulables par notre modèle.

In [ ]:
# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

In [ ]:
# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

Les données de test nous permettront de tester notre modèle sur des données qu'il n'a pas utilisé pour se construire. Cela nous permet de tester notre modèle sur de "nouvelles données". C'est son pouvoir de généralisation qui nous intéresse.

**<span style='color:green'> Évaluation d'un modèle</span>** 
Il n'existe pas une seule manière d'évaluer les performances d'un modèle. En voici quelques-une.

*Précision*

Considérons une tâche de classification ($y=\{1,\ldots, C\}$), la précision d'un modèle $h$ pour la classe $c$ est

$$\text{prec}_c(h)=\frac{\sum_i \textbf{1}\{y_i=c, h(x_i)=y_i\}}{\sum_i \textbf{1}\{h(x_i)=c\}}$$

On veut que le score soit proche de $1$.

---

*Rappel*

Considérons une tâche de classification ($y=\{1,\ldots, C\}$), le rappel d'un modèle $h$ pour la classe $c$ est

$$\text{rec}_c(h)=\frac{\sum_i \textbf{1}\{y_i=c, h(x_i)=y_i\}}{\sum_i \textbf{1}\{y_i=c\}}$$

On veut que le score soit proche de $1$.

---

*Score f1*

Ce dernier combine le rappel et la précision&nbsp;:

$$\text{f1}_c(h)=2\frac{\text{prec}_c(h)\cdot\text{rec}_c(h)}{\text{prec}_c(h)+\text{rec}_c(h)}$$

On veut que le score soit proche de $1$.

---

*L'accuracy*

C'est une sorte de généralisation de la précision&nbsp;:

$$\text{acc}(h)=\frac{\sum_i\textbf{1}\{h(x_i)=y_i\}}{n},$$

c'est le ratio de bonnes prédictions.

---

*Le support*

C'est le nombre de points de notre jeu de données qui sont concernées par la métrique.



 ----

In [ ]:
model = Memorize()
model.fit(X_train, y_train)

On commence par tester les performances de notre modèle sur notre jeu d'apprentissage.

In [ ]:
predicted = model.predict(X_train)

In [ ]:
print(f'Classification report for classifier Memorize on train:\n'
      f'{metrics.classification_report(y_train, predicted)}\n')

Notre modèle est parfait ! Aucune erreur. On ne peut pas faire mieux ! Et du côté du test ?

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print(f'Classification report for classifier Memorize on test:\n'
      f'{metrics.classification_report(y_test, predicted)}\n')

C'est ridiculement mauvais : on se trompe neuf fois sur dix, soit exactement ce qu'on attendrait d'une réponse aléatoire.

Oui mais on a fait exprès de construire le modèle de cette manière ! En réalité, il existe une infinité de fonctions, paramétriques ou non, qu'on peut rendre aussi bonne qu'on veut sur nos données mais qui seraient particulièrement mauvaises sur de nouvelles données (cela inclut les modèles usuels et c'est pour cela qu'on a besoin d'experts !)... Toute la difficulté du *machine learner* va être de contrôler cela.

**<span style='color:blue'> $\texttt{memorize}$ en pratique</span>** 
Cette approche n'est pas totalement absurde. Si $|\mathcal{X}|<\infty$, ou si $\mathcal{X}$ est discret, alors plus on collectera de données plus les nouvelles données auront déjà été vues et nos prédictions deviendront intéressantes. C'est exactement ce que nous faisons face à un nouveau paquet de bonbons dont nous ne connaissons pas le goût.



 ----

## III. Une autre première approche logique&nbsp;: le KNN (AS)

Intuitivement, on a envie de dire que nos données ne sont pas complètement déstructurées. Deux clients très similaires achèteront très probablement des produits très similaires. Un _trois_ ressemble plus à un _trois_ qu’à un _cinq_ et un _cinq_ ressemble plus à un _cinq_ qu’à un _trois_. Finalement, on généralise un petit peu l'exemple précédent. Au lieu de répondre aléatoirement si je ne connais pas la donnée, je cherche l'exemple le plus proche et je prédis le même label ! Plus rigoureusement, notre modèle de prédiction fonctionne comme suit&nbsp;: 


$$\hat{y}_\text{new}=y\text{ avec }(x, y)=\text{argmin}_{(x, y)\in S}\lVert x-x_{\text{new}}\rVert_2.$$

On peut imaginer que si plusieurs points sont équidistants, la réponse se fait aléatoirement entre les labels possibles. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

**<span style='color:blue'> Exercice</span>** 
**Utilisez l'objet $\texttt{KNeighborsClassifier}$ avec le paramètre $\texttt{n}\_\texttt{neighbors=1}$ et entraînez le sur $\texttt{X}\_\texttt{train}$.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
model = 
model.fit(X_train, y_train)
###############################################################

predicted = model.predict(X_train)

print(f'Classification report for classifier {model} on train:\n'
      f'{metrics.classification_report(y_train, predicted)}\n')


On a testé le modèle sur le jeu d'apprentissage et on est toujours aussi bon sur le jeu d'apprentissage ! Cependant, c'est attendu car l'image qui ressemble le plus à une autre est l'image elle-même. Prédisons maintenant sur le test.

**<span style='color:green'> La matrice de confusion</span>** 
Les métriques précédentes nous donnent les performances du modèle en moyenne ainsi que par classe. Cependant, cela ne nous donne que très peu d'information quant au type d'erreur qu'il fait. La **matrice de confusion** permet de comprendre le type d'erreurs que fait notre modèle. La cellule $i,j$ indique le nombre d'éléments de la classe $i$ qui ont été prédits de la classe $j$.


 ----

In [ ]:
predicted = model.predict(X_test)

print(f'Classification report for classifier {model} on test:\n'
      f'{metrics.classification_report(y_test, predicted)}\n')

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()




Is Machine learning solved ? Minute papillon ! Ce modèle est très sensible au bruit ! Supposons qu’une de nos données soit bruitée (e.g. un 3 qui ressemble à un 8). Si une nouvelle donnée représentant un $8$ se retrouve à côté de cette anomalie, elle sera mal prédite. Nous pouvons adresser cette limite de la manière suivante&nbsp;: au lieu de regarder le point le plus proche, on regarde les $k$ points les plus proches et on fait un vote à la majorité. Plus formellement la prédiction est faite comme suit&nbsp;:

$$\hat{y}_\text{new}=\text{majority$\_$voting}(\texttt{KNN}.\texttt{labels})\text{  où  }\texttt{KNN}=\text{argmin}_{S^\prime\subset S,\ |S^\prime|=K}\sum_i \lVert x_i- x_{\text{new}}\rVert.$$

Dans le cas où on chercherait à faire une régression, on remplace le vote à la majorité par une moyenne.

Récupérons le jeu de données $\texttt{iris}$.

In [ ]:
iris = datasets.load_iris()
# décommentez la ligne suivante pour obtenir des informations
# sur le dataset iris.
# print(iris.DESCR)

De la même manière que précédemment, on construit notre jeu d'apprentissage pour construire notre modèle et notre jeu de test pour en tester les performances.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.5, shuffle=True
)

**<span style='color:blue'> Exercice</span>** 
**Utilisez l'objet $\texttt{KNeighborsClassifier}$ avec le paramètre $\texttt{n}\_\texttt{neighbors=1}$ et entraînez le sur $\texttt{X}\_\texttt{train}$. Faites une prédiction sur $\texttt{predicted=X}\_\texttt{test}$.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
model = ...
model.fit(...

predicted = ... 
###############################################################

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()


Les performances sont déjà très bonnes ! Mais il est possible de gagner un tout petit peu de performances en considérant plus de voisins :

**<span style='color:blue'> Exercice </span>** **Utilisez l'objet $\texttt{KNeighborsClassifier}$ avec le paramètre $\texttt{n}\_\texttt{neighbors=10}$ et entraînez le sur $\texttt{X}\_\texttt{train}$. Faites une prédiction sur $\texttt{predicted=X}\_\texttt{test}$.**



 ----

In [ ]:
####### Complete this part ######## or die ####################
model = ...
model.fit(...

predicted = ...
###############################################################

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()


## IV. Les arbres de décision ou Classification and Regression Tree (CART) (AS)

De la même manière que pour l'algorithme KNN, on supposera ici que nos données admettent une certaine structure et que des points proches possèdent probablement le même label ou une prediction proche dans le cas de la régression. Ici, à la différence du KNN, la notion de voisinage se construit au travers d'hyperrectangles parallèles aux axes. Afin de bien comprendre le fonctionnement, supposons que notre arbre de décision soit déjà construit et soit celui représenté par la figure ci-dessous. Prenons une nouvelle donnée&nbsp;:

$$x_{\text{new}}=[\text{ecoute:}1, \text{math:}12,\text{info:}18]^T,$$ 


et partons de la racine de notre arbre. Cette racine possède deux branches sortantes. Le choix de la branche se fait à partir d'un critère sur une des variables explicatives de $x$. Dans notre exemple la variable explicative est l'écoute en cours. Si l'étudiant écoute, on prend la branche de droite, sinon celle de gauche. Dans notre, ce sera donc la branche de droite. La nouvelle variable à regarder est la note de math et sa valeur doit être supérieure à 10. C'est notre cas et nous prenons la branche de droite. Nous sommes à une feuille et la prédiction à faire est "oui, l'étudiant s'en sortira".

![Decision tree](https://raw.githubusercontent.com/maximiliense/lmiprp/main/Travaux%20Pratiques/Machine%20Learning/Introduction/data/Introduction/cart.jpg)



Le choix de la règle de décision à chaque noeud peut être adapté afin d'obtenir des régions à la géométrie variable. La construction d'un arbre se fait en partant de la racine vers les feuilles et en choisissant intérativement les variables explicatives qui ont le plus d'effet sur notre prédiction (via diverses critères).

**<span style='color:blue'> Les feuilles de l'arbre</span>** 
En pratique, les feuilles de notre arbre contiennent les exemples d'apprentissage de notre jeu de données. La prédiction dépend des exemples contenus dans la feuille qui nous concerne.



 ----

La séquence suivante abordera plus en détails les arbres de classification et de régression.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

**<span style='color:blue'> Exercice</span>** **Utilisez l'objet $\texttt{DecisionTreeClassifier}$ avec le paramètre $\texttt{max}\_\texttt{depth=2}$ et entraînez le sur $\texttt{X}\_\texttt{train}$. Faites une prédiction sur $\texttt{predicted=X}\_\texttt{test}$.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
model = ...
model.fit(...

predicted = ...
###############################################################

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()


## V. Les forêts aléatoires ou Random Forest (RF) (AS)

Les arbres de décision peuvent être sujets au surapprentissage. Une manière de compenser le problème est d'en construire plusieurs où chaque arbre est construit en ne voyant qu'une partie des données. Enfin leurs prédictions sont aggrégées. Ces approches sont généralement beaucoup plus performantes que les arbres simples. Malheureusement, autant avec un arbre simple on pouvait essayer de comprendre la prédiction, autant ici, cela devient difficile.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**<span style='color:blue'> Exercice</span>** **Utilisez l'objet $\texttt{RandomForestClassifier}$ et entraînez le sur $\texttt{X}\_\texttt{train}$. Faites une prédiction sur $\texttt{predicted=X}\_\texttt{test}$.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
###############################################################

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()


## VI. Choix des hyperparamètres (AS)

Pour les modèles précédents, nous avons dû choisir différents paramètres qui affectaient les performances de notre modèle. Nous les avons choisis en regardant les performances de notre modèle sur le jeu de test. Cependant, les bonnes performances étaient peut-être un coup de chance !

Il existe deux stratégies d'évaluation sans biais de la qualité de notre modèle&nbsp;:
* La validation non croisée où une partie de notre jeu de données est cachée pendant l'apprentissage puis utilisée afin d'évaluer les performances du modèle. Il s'agit du découpage train/test. Cette stratégie est un estimateur sans biais de la qualité de notre modèle mais possède une variance plus forte que la validation croisée. Elle peut-être particulièrement utile lorsque le coup d'apprentissage d'un modèle est très élevé (e.g. *deep learning*)
* La validation croisée où notre jeu de données est divisé en *k* parties (on parle aussi de *k-fold*). Évidemment, $k\in\{2, ..., n\}$ où $n$ est la taille du jeu de données. Chacune des parties jouera successivement le rôle de jeu de test pendant que les $k-1$ autres parties serviront à calculer notre modèle. Le résultat de cette procédure est un vecteur de $k$ scores dont on peut calculer la moyenne, la variance, etc.


On peut illustrer la méthode des *k-folds* via l'exemple suivant&nbsp;:

$$\begin{align*}
\text{Appartient au train set: } \color{red}{\boxed{}}&\text{ et appartient au test set: }\color{green}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 1: }\color{green}{\boxed{}}\color{red}{\boxed{}}&\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 2: }\color{red}{\boxed{}}\color{green}{\boxed{}}&\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 3: }\color{red}{\boxed{}}\color{red}{\boxed{}}&\color{green}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 4: }\color{red}{\boxed{}}\color{red}{\boxed{}}&\color{red}{\boxed{}}\color{green}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 5: }\color{red}{\boxed{}}\color{red}{\boxed{}}&\color{red}{\boxed{}}\color{red}{\boxed{}}\color{green}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 6: }\color{red}{\boxed{}}\color{red}{\boxed{}}&\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{green}{\boxed{}}\color{red}{\boxed{}}
\end{align*}$$

$$\begin{align*}
\text{Step 7: }\color{red}{\boxed{}}\color{red}{\boxed{}}&\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{red}{\boxed{}}\color{green}{\boxed{}}
\end{align*}$$

**<span style='color:orange'> Overfitter à la main</span>** 
Attention, quand on fait quelques tests à la main et que l'on évalue nos performances sur le jeu de test, on est déjà entrain de faire de la sélection de modèle. On se sert alors du test comme d’un ensemble de validation. C’est d’autant plus vrai lorsqu’on a peu de données d'apprentissage.


 ----
La méthode $\texttt{cross_val_score}$ de $\texttt{sklearn}$ permet de réaliser cette procédure. On pourra renseigner le paramètre $\texttt{cv}$ qui indique le nombre $k$ et le paramètre $\texttt{scoring}$ qui donne la métrique que l'on souhaite calculer.

Si on cherche à trouver une valeur d’un hyper-paramètre du modèle, l’objet $\texttt{𝙶𝚛𝚒𝚍𝚂𝚎𝚊𝚛𝚌𝚑𝙲𝚅}$ applique une validation croisée en cherchant différentes valeurs de cet hyper-paramètre.


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'max_depth': [2, 3, 4, 5, 10, None]
}

**<span style='color:blue'> Exercice</span>** **Utilisez l'objet $\texttt{GridSearchCV}$ pour faire une recherche par grille sur le modèle $\texttt{RandomForestClassifier}$ et entraînez le sur $\texttt{X}\_\texttt{train}$.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
###############################################################

fig, ax = plt.subplots(figsize=(12, 8))
disp = metrics.plot_confusion_matrix(model, X_test, y_test, ax=ax)
disp.figure_.suptitle("Confusion Matrix")
plt.show()


Notre RandomForest de base était déjà très bon !

## VII. L'algorithme des K-Moyennes (ANS)

Il s'agit ici d'un algorithme non supervisé. Imaginons que nous ayons collecté un jeu de données $S_n=\{(X_i)\}_{i\leq n}$. On sait qu'il existe des groupes dans nos données. Supposons même qu'on sâche qu'il existe $K$ groupes. L'idée de l'algorithme des K-Moyennes va être de détecter ces $K$ groupes en trouvant une solution au problème d'optimisation suivant&nbsp;: 

$$\text{KMeans}=\text{argmin}_{m_1, \ldots, m_K\in\mathcal{X}, c_1,\ldots,c_n\in\{1,\ldots,K\}}\sum_{i=1}^K\sum_{j=1}^n\textbf{1}\{c_j=i\}\lVert m_i-x_j\rVert_2=\text{argmin}_{m_1, \ldots, m_K\in\mathcal{X}, c_1,\ldots,c_n\in\{1,\ldots,K\}}\sum_{j=1}^n\lVert x_j-m_{c_j}\rVert_2.$$

Dit autrement, chaque groupe est représenté par une coordonnée $m_i$ (qui s’avère être la moyenne des éléments du groupe) et chaque élément de notre jeu de données n’est associé qu’à un seul groupe $c_j$.

Considérons le jeu de données suivant.

In [ ]:
import numpy as np

def sample_data(n):
    means = np.array([[-0.5, 0.5], [1, 1], [0.5, -0.5]])
    cov = np.diag([1, 1])/15
    X = np.concatenate([
        np.random.multivariate_normal(m, cov, size=n) for m in means
    ], axis=0)
    y = [i//n for i in range(3*n)]
    return X, y
    
X, y = sample_data(10)

In [ ]:
import matplotlib.pyplot as plt
def plot_clusters(X, c, means=None, path=None):
    plt.figure(figsize=(12, 8))
    plt.axis('off')
    plt.scatter(X[:, 0], X[:, 1], c=c)
    plt.title('Nos données et leur label inconnu')
    if means is not None:
        for m in means:
            plt.scatter([m[0]], [m[1]], s=55, color='red')
    if path is not None:
        for p in path:
            plt.plot(p[:, 0], p[:, 1], color='red')
    plt.show()
plot_clusters(X, y)

Le problème de K-Means est NP-Difficile. Pour cela, nous utilisons en pratique un heuristique appelé "algorithme de LLoyd" qui fonctionnde la manière suivante&nbsp;:

1.  On initialise les k moyennes
2.  On assigne tous nos points à leur moyenne la plus proche
3.  On met à jour les moyennes avec les nouveaux points
4.  Si le déplacement des moyennes est significatif, on reprend à l'étape 2

**<span style='color:green'> Fixer le $k$</span>** 
Le $k$ peut être fixé via une selection de paramètres et une validation croisée.



 ----

**<span style='color:blue'> Exercice</span>** **Le problème est NP-Difficile et un algorithme permettant de le résoudre est l'algorithme de LLoyd. Implémentez le.**


 ----

In [ ]:
class KMeans(object):
    def __init__(self, k=3, epsilon=0.001):
        self.k = k
        self.epsilon = epsilon
    
    def fit(self, X):
        ####### Complete this part ######## or die ####################
        # K means initialization
        ...
        # Step 1
        ...
        # Step 2
        ...
        ###############################################################
        return assignment, means, path
            

model = KMeans(3)
plot_clusters(X, *model.fit(X))


On se rend compte qu'on arrive à retrouver les 3 groupes automatiquement (Les couleurs sont celles calculées par notre modèle des K-Moyennes) !

## VIII. La malédiction de la dimension

Nous avons pu observer des scénarios où l'erreur sur notre jeu de données d'apprentissage était $0$ alors que notre modèle n'était pas si bon que cela sur notre jeu de test. Cet écart peut même devenir catastrophique ! De manière plus rigoureuse, le gap de généralisation de notre estimateur $\hat{h}$ est la quantité suivante&nbsp;:


$$\text{gap}(\hat{h})=|Re(\hat{h})-R(\hat{h})|.$$


Où $Re$ fait référence à notre risque empirique, c'est-à-dire l'erreur sur le jeu d'apprentissage et $R$ à l'erreur en espérance.

Il est possible d'avoir une idée de $R(\hat{h})$ en passant par un jeu de test ou par une autre stratégie d'évaluation via un jeu de test par exemple, comme nous avons pu le voir. Nous allons ici nous rendre compte que les modèles qui regardent le voisinage de nos données souffrent d'une grosse limite liée à ce qu'on appelle *la malédiction de la dimension* et qui affecte grandement ce *gap*.

### En détails

La malédiction de la dimension fait référence aux résultats contre-intuitifs qui apparaissent lorsque la dimension augmente. 

**<span style='color:blue'> Une illustration avec l'orange multi-dimensionnelle</span>** 
Cette exemple donne une image plus visuelle de la malédiction de la dimension. Modélisons une orange comme une boule parfaite de rayon $8\text{cm}$ avec une enveloppe (i.e. la peau) d'une épaisseur de $5\text{mm}$ partout. Le volume d'une boule $\mathcal{B}_r$ de rayon $r$ dans un espace de dimension $d$ est donné par&nbsp;:

$$\text{vol}(\mathcal{B}_r)=K r^d,$$

où $K$ est une constante qui dépend de la dimension. La question qu'on peut se poser est celle du volume occupé par la peau de l'orange relativement au volume total de l'orange. C'est le volume total de l'orange auquel on soustrait le volume de l'orange sans la peau divisé par le volume total de l'orange&nbsp;:

$$\text{ratio}=\frac{\text{vol}(\mathcal{B}_8)-\text{vol}(\mathcal{B}_{8-0.05})}{\text{vol}(\mathcal{B}_8)}.$$

En dimension $3$, cela nous donne $\text{ratio}=0.018$. Moins de $2\%$ du volume de l'orange est occupé par la peau. Que se passe-t-il en dimension 500 ? On obtient $\text{ratio}\approx 96\%$. Presque tout le volume de l'orange est occupé par la peau ! Et $500$ ne représente pas la très grande dimension (une image en couleur de 200px$\times$200px a une dimension de 120000).



 ----

Une première manière de l'observer en *machine learning* est possible grâce au KNN. Ce dernier classe un nouvel élément en fonction de ses voisins dans le jeu d'apprentissage. Nous allons en particulier étudier l'évolution du risque de généralisation en fonction de la dimension. Plus précisément, les données synthétiques sont construites de la manière suivante&nbsp;:

In [ ]:
import numpy as np
def sample_data(n, k=3, d=3, mu=1):
    y = np.random.randint(0, 2, size=(n, 1))
    
    X = np.random.normal(mu, 1, size=(n, k))
    X = y*X-(1-y)*X # positive have mean mu and negative, -mu
    noise = np.random.normal(0, 1, size=(n, d-k))
    X = np.concatenate([X, noise], axis=1)
    
    return X, y

Dit autrement, $k$ dimensions contiennent le signal intéressant pour notre tâche et $d$ dimensions ne servent à rien. Nous observons ci-dessous ce qui se passe lorsqu'on rajouter des dimensions de bruits (i.e. qui ne servent à rien). C'est typiquement ce pourrait se passer avec des images. Une photo de chien ne contient pas que des pixels descriptifs du concept de chien.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

scores = []
redo = 5
max_dim = 5000
first_dim = 10
steps = 100

for d in range(first_dim, max_dim, steps):
    s = 0
    for _ in range(redo):
        X, y = sample_data(100, d=d)
        X_test, y_test = sample_data(200, d=d)
        c = KNeighborsClassifier()
        c.fit(X, y.reshape((y.shape[0],)))
        s += c.score(X_test, y_test.reshape((y_test.shape[0],)))/redo
    scores.append(s)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# configuration generale de matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('ggplot')

plt.plot(list(range(first_dim, max_dim, steps)), scores)
plt.title('Evolution de l\'accuracy en fonction de la dimension du probleme')
plt.show()

**<span style='color:blue'> Exercice</span>** 
**Quelle est l'accuracy d'un classifieur aléatoire ?**


 ----
**<span style='color:blue'> Exercice</span>** **Expliquez pourquoi l'accuracy diminue lorsqu'on rajoute des dimensions sans signal.**


 ----

De manière similaire, étudions l'évolution des distances lorsque la dimension évolue. Nous allons tirer aléatoirement un ensemble de vecteurs dans $\mathbb{R}^d$ et nous calculerons la norme $\lVert x\rVert^2_2$ moyenne, maximale et minimale de notre tirage. De la même manière nous calculerons la distance $\lVert x-y\rVert_2$ moyenne, maximale et minimale entre les couples de points de notre jeu de données. Enfin, l'objectif sera d'étudier ces quantités en faisant évoluer la dimension $d$ du problème.

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix

def sample_data(n, d):
    return np.random.uniform(-1, 1, size=(n, d))/np.sqrt(d)
X = sample_data(100, 10)

In [ ]:
redo = 50
def experiment_(d):
    min_ = 0
    max_ = 0
    mean_ = 0
    for _ in range(redo):
        X = sample_data(100, d)
        vec = np.sqrt((X**2).sum(axis=1))
        min_ += vec.min()/redo
        max_ += vec.max()/redo
        mean_ += vec.mean()/redo
        
        mat = distance_matrix(X[:25], X[:25])
        triu = np.triu(mat)
        triu = triu[triu!=0]
        dist_max = triu.max()
        dist_min = triu.min()
        dist_mean = triu.mean()
    return min_, max_, mean_, dist_min, dist_max, dist_mean
idx = []
val = []
for d in range(10, 1000, 100):
    idx.append([d])
    val.append(experiment_(d))
for d in range(2000, 10000, 1000):
    idx.append([d])
    val.append(experiment_(d))
arr = np.concatenate([np.array(idx), np.array(val)], axis=1)

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.plot(arr[:, 0], arr[:, 1], label='Min')
plt.plot(arr[:, 0], arr[:, 2], label='Max')
plt.plot(arr[:, 0], arr[:, 3], label='Moy')
plt.xlabel('Dimension du problème')
plt.ylabel('Norme $\ell_2$ des vecteurs')
plt.legend()
plt.title('Evolution des normes')
plt.subplot(1, 2, 2)
plt.plot(arr[:, 0], arr[:, 4], label='Min')
plt.plot(arr[:, 0], arr[:, 5], label='Max')
plt.plot(arr[:, 0], arr[:, 6], label='Moy')
plt.xlabel('Dimension du problème')
plt.ylabel('Distance $\ell_2$ entre nos vecteurs')
plt.legend()
plt.title('Evolution des distances')

**<span style='color:blue'> Exercice</span>** 
**Quel phénomène mathématique pouvons nous invoquer afin d'expliquer cela ?**



 ----
Soit $x_\text{new}$ une nouvelle donnée. Une petite perturbation du point de notre jeu d'apprentissage le plus différent de $x_\text{new}$ peut le transformer en le point le plus proche est inversement... C'est une grosse limite des modèles précédents. Il faut soit réfléchir à réduire la dimension, soit injecter de la connaissance dans nos modèles, etc.

## X. Quelques modèles proposés par $\texttt{sklearn}$

Voici une liste (absolument non exhaustive) de quelques modèles proposés par la librairie $\texttt{sklearn}$. Certains (pas nécessairement présents dans cette liste) seront approfondis dans les prochaines séquences.

*  Supervisé&nbsp;:
 * Régression dans $\mathbb{R}$&nbsp;:
    * [Régression linéaire](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) et ses variantes&nbsp;:
      * [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
      * [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
      * [Elastic-Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)
    * [Régression KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
    * [Arbre de régression](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)
    * [Forêts aléatoires](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
    * etc.
  * Classification (ou assimilés)&nbsp;:
    * [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
    * [Classification KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
    * [Régression Logistique](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
    * [Arbre de classification](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
    * [Forêts aléatoires](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* Non-supervisé&nbsp;:
  * [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) (clustering)
  * [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) (apprentissage de *manifold*)
  * [OneclassSVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html) (détection de nouveauté)
  
  
  
Pour une liste plus exhaustive des différents outils $\texttt{sklearn}$ merci de consulter le lien suivant&nbsp; :
* https://scikit-learn.org/stable/supervised_learning.html.

## XI. Le *no-free-lunch* Theorem

Nous avons vu plus haut que le modèle $\texttt{memorize}$ pouvait être parfaitement bon sur les données d'apprentissage mais échouer sur de nouvelles données : il ne généralise pas. Pour cela, nous avons rajouté certaines hypothèses et construit de nouveaux modèles comme le $\texttt{KNN}$ qui exploite le voisinage d'un point pour pouvoir faire une prédiction. La malédiction de la dimension, malheureusement, pénalise ce fonctionnement dès que la dimension de l'espace d'entrée devient trop grande.

Une question que nous pouvons nous poser est la suivante : pouvons nous construire une règle de classification générique, optimisée à partir d'un jeu de données, qui puisse offrir la garantie de toujours fonctionner ?

La réponse est non, et au-delà de cela, pour chaque modèle que nous allons utiliser, nous devrons faire des hypothèses sur les données. Dans les cas où ces hypothèses seraient falsifiées alors notre stratégie échouera.

**<span style='color:blue'> *No-free-lunch Theorem*</span>** Soit $\mathcal{D}_n=\{(X_i, Y_i)\}_{i\leq n}$ un jeu de données et $\hat{h}_n:\mathcal{X}\mapsto\mathcal{Y}$ un modèle de classification (un classifieur) construit à partir de $\mathcal{D}$ selon une règle au choix. L'indice $n$ montre la dépendence sur la taille de notre jeu de données. Soit $1/16\geq\{a_n\}_{n>0}>0$ une suite qui converge vers $0$ mais à une vitesse aussi lente qu'on le veuille. Alors, il existe un problème (i.e. une distribution sur $X\times Y$) tel que le meilleur classifieur $h^\star$ fasse $0$, mais que notre estimateur ait une erreur satisfaisant l'inégalité&nbsp;:

$$\mathbb{E}\big[R(h_n)\big]>a_n.$$


 ----
Dit autrement, plus le jeu de données sera grand plus l'erreur sera faible. Mais cette décroissance de l'erreur peut être arbitrairement lente.